## NYC Property Price Prediction

Given *data about property in New York City*, let's try to predict the **price** of a given piece of property.

We will use XGBoost to make our predictions.

Data source: https://www.kaggle.com/datasets/new-york-city/nyc-property-sales

### Importing Libraries

In [19]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.metrics import r2_score

In [20]:
data = pd.read_csv('archive/nyc-rolling-sales.csv')
data

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

### Preprocessing

In [22]:
df = data.copy()

In [23]:
df

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00


In [24]:
df['SALE PRICE'].unique()

array(['6625000', ' -  ', '3936272', ..., '408092', '11693337', '69300'],
      shape=(10008,), dtype=object)

In [25]:
# Remove any records where we do not have a sale price
df['SALE PRICE'] = df['SALE PRICE'].replace(' -  ', np.nan).astype(float)

In [26]:
df.isna().mean()

Unnamed: 0                        0.000000
BOROUGH                           0.000000
NEIGHBORHOOD                      0.000000
BUILDING CLASS CATEGORY           0.000000
TAX CLASS AT PRESENT              0.000000
BLOCK                             0.000000
LOT                               0.000000
EASE-MENT                         0.000000
BUILDING CLASS AT PRESENT         0.000000
ADDRESS                           0.000000
APARTMENT NUMBER                  0.000000
ZIP CODE                          0.000000
RESIDENTIAL UNITS                 0.000000
COMMERCIAL UNITS                  0.000000
TOTAL UNITS                       0.000000
LAND SQUARE FEET                  0.000000
GROSS SQUARE FEET                 0.000000
YEAR BUILT                        0.000000
TAX CLASS AT TIME OF SALE         0.000000
BUILDING CLASS AT TIME OF SALE    0.000000
SALE PRICE                        0.172222
SALE DATE                         0.000000
dtype: float64

In [27]:
df = df.dropna(axis=0).reset_index(drop=True)

In [28]:
df['SALE PRICE'].isna().sum()

np.int64(0)

In [29]:
df

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000.0,2017-07-19 00:00:00
1,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272.0,2016-09-23 00:00:00
2,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000.0,2016-11-17 00:00:00
3,10,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,,C4,210 AVENUE B,,10009,8,0,8,1750,4226,1920,2,C4,3192840.0,2016-09-23 00:00:00
4,13,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,,D9,629 EAST 5TH STREET,,10009,24,0,24,4489,18523,1920,2,D9,16232000.0,2016-11-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000.0,2016-11-28 00:00:00
69983,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000.0,2017-04-21 00:00:00
69984,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000.0,2017-07-05 00:00:00
69985,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337.0,2016-12-21 00:00:00


In [31]:
df['BLOCK'].unique()

array([ 392,  402,  404, ..., 6967, 7074, 7092], shape=(10848,))

In [32]:
df.columns

Index(['Unnamed: 0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [35]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'EASE-MENT': 1,
 'BUILDING CLASS AT PRESENT': 161,
 'ADDRESS': 57304,
 'APARTMENT NUMBER': 3320,
 'LAND SQUARE FEET': 5293,
 'GROSS SQUARE FEET': 5095,
 'BUILDING CLASS AT TIME OF SALE': 161,
 'SALE DATE': 362}

In [36]:
# Remove unnecessary/difficult feature columns
df = df.drop(['Unnamed: 0', 'BLOCK', 'ADDRESS', 'LOT', 'EASE-MENT', 'APARTMENT NUMBER'], axis=1)

In [37]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BUILDING CLASS AT PRESENT': 161,
 'LAND SQUARE FEET': 5293,
 'GROSS SQUARE FEET': 5095,
 'BUILDING CLASS AT TIME OF SALE': 161,
 'SALE DATE': 362}

In [45]:
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace(' -  ', np.nan).astype(float)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace(' -  ', np.nan).astype(float)

In [46]:
df.isna().mean()

BOROUGH                           0.000000
NEIGHBORHOOD                      0.000000
BUILDING CLASS CATEGORY           0.000000
TAX CLASS AT PRESENT              0.000000
BUILDING CLASS AT PRESENT         0.000000
ZIP CODE                          0.000000
RESIDENTIAL UNITS                 0.000000
COMMERCIAL UNITS                  0.000000
TOTAL UNITS                       0.000000
LAND SQUARE FEET                  0.302742
GROSS SQUARE FEET                 0.310615
YEAR BUILT                        0.000000
TAX CLASS AT TIME OF SALE         0.000000
BUILDING CLASS AT TIME OF SALE    0.000000
SALE PRICE                        0.000000
SALE DATE                         0.000000
dtype: float64

In [47]:
# Fill missing values with column means
for column in ['LAND SQUARE FEET', 'GROSS SQUARE FEET']:
    df[column] = df[column].fillna(df[column].mean())

In [48]:
df.isna().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BUILDING CLASS AT PRESENT         0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64

In [53]:
# Get year, month and day features from SALE DATE column
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

df['YEAR'] = df['SALE DATE'].apply(lambda x: x.year)
df['MONTH'] = df['SALE DATE'].apply(lambda x: x.month)
df['DAY'] = df['SALE DATE'].apply(lambda x: x.day)

df = df.drop('SALE DATE', axis=1)

In [54]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,YEAR,MONTH,DAY
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,5,0,5,1633.0,6440.0,1900,2,C2,6625000.0,2017,7,19
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016,9,23
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,6,0,6,2369.0,4615.0,1900,2,C2,8000000.0,2016,11,17
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016,9,23
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,D9,10009,24,0,24,4489.0,18523.0,1920,2,D9,16232000.0,2016,11,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2400.0,2575.0,1998,1,B9,450000.0,2016,11,28
69983,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2498.0,2377.0,1998,1,B9,550000.0,2017,4,21
69984,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B2,10309,2,0,2,4000.0,1496.0,1925,1,B2,460000.0,2017,7,5
69985,5,WOODROW,22 STORE BUILDINGS,4,K6,10309,0,7,7,208033.0,64117.0,2001,4,K6,11693337.0,2016,12,21


In [55]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BUILDING CLASS AT PRESENT': 161,
 'BUILDING CLASS AT TIME OF SALE': 161}

In [56]:
# Make numeric categorical features into string columns
for column in ['BOROUGH', 'ZIP CODE']:
    df[column] = df[column].astype('str')

In [57]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'BOROUGH': 5,
 'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BUILDING CLASS AT PRESENT': 161,
 'ZIP CODE': 184,
 'BUILDING CLASS AT TIME OF SALE': 161}

In [58]:
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix, dtype=int)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [59]:
df.select_dtypes('object').columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BUILDING CLASS AT PRESENT', 'ZIP CODE',
       'BUILDING CLASS AT TIME OF SALE'],
      dtype='object')

In [60]:
# One hot encode remaining categorical features
df = onehot_encode(df, df.select_dtypes('object').columns, ['BR', 'NB', 'BCC', 'TCP', 'BCP', 'ZC', 'BCTS'])

In [61]:
df

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,SALE PRICE,YEAR,MONTH,DAY,BR_1,BR_2,BR_3,BR_4,BR_5,NB_AIRPORT LA GUARDIA,NB_ALPHABET CITY,NB_ANNADALE,NB_ARDEN HEIGHTS,NB_ARROCHAR,NB_ARROCHAR-SHORE ACRES,NB_ARVERNE,NB_ASTORIA,NB_BATH BEACH,NB_BATHGATE,NB_BAY RIDGE,NB_BAYCHESTER,NB_BAYSIDE,NB_BEDFORD PARK/NORWOOD,NB_BEDFORD STUYVESANT,NB_BEECHHURST,NB_BELLE HARBOR,NB_BELLEROSE,NB_BELMONT,NB_BENSONHURST,NB_BERGEN BEACH,NB_BLOOMFIELD,NB_BOERUM HILL,NB_BOROUGH PARK,NB_BRIARWOOD,NB_BRIGHTON BEACH,NB_BROAD CHANNEL,NB_BRONX PARK,NB_BRONXDALE,NB_BROOKLYN HEIGHTS,NB_BROWNSVILLE,NB_BULLS HEAD,NB_BUSH TERMINAL,NB_BUSHWICK,NB_CAMBRIA HEIGHTS,NB_CANARSIE,NB_CARROLL GARDENS,NB_CASTLE HILL/UNIONPORT,NB_CASTLETON CORNERS,NB_CHELSEA,NB_CHINATOWN,NB_CITY ISLAND,NB_CITY ISLAND-PELHAM STRIP,NB_CIVIC CENTER,NB_CLINTON,NB_CLINTON HILL,NB_CLOVE LAKES,NB_CO-OP CITY,NB_COBBLE HILL,NB_COBBLE HILL-WEST,NB_COLLEGE POINT,NB_CONCORD,NB_CONCORD-FOX HILLS,NB_CONEY ISLAND,NB_CORONA,NB_COUNTRY CLUB,NB_CROTONA PARK,NB_CROWN HEIGHTS,NB_CYPRESS HILLS,NB_DONGAN HILLS,NB_DONGAN HILLS-COLONY,NB_DONGAN HILLS-OLD TOWN,NB_DOUGLASTON,NB_DOWNTOWN-FULTON FERRY,NB_DOWNTOWN-FULTON MALL,NB_DOWNTOWN-METROTECH,NB_DYKER HEIGHTS,NB_EAST ELMHURST,NB_EAST NEW YORK,NB_EAST RIVER,NB_EAST TREMONT,NB_EAST VILLAGE,NB_ELMHURST,NB_ELTINGVILLE,NB_EMERSON HILL,NB_FAR ROCKAWAY,NB_FASHION,NB_FIELDSTON,NB_FINANCIAL,NB_FLATBUSH-CENTRAL,NB_FLATBUSH-EAST,NB_FLATBUSH-LEFFERTS GARDEN,NB_FLATBUSH-NORTH,NB_FLATIRON,NB_FLATLANDS,NB_FLORAL PARK,NB_FLUSHING MEADOW PARK,NB_FLUSHING-NORTH,NB_FLUSHING-SOUTH,NB_FORDHAM,NB_FOREST HILLS,NB_FORT GREENE,NB_FRESH KILLS,NB_FRESH MEADOWS,NB_GERRITSEN BEACH,NB_GLEN OAKS,NB_GLENDALE,NB_GOWANUS,NB_GRAMERCY,NB_GRANT CITY,NB_GRASMERE,NB_GRAVESEND,NB_GREAT KILLS,NB_GREAT KILLS-BAY TERRACE,NB_GREENPOINT,NB_GREENWICH VILLAGE-CENTRAL,NB_GREENWICH VILLAGE-WEST,NB_GRYMES HILL,NB_HAMMELS,NB_HARLEM-CENTRAL,NB_HARLEM-EAST,NB_HARLEM-UPPER,NB_HARLEM-WEST,NB_HIGHBRIDGE/MORRIS HEIGHTS,NB_HILLCREST,NB_HOLLIS,NB_HOLLIS HILLS,NB_HOLLISWOOD,NB_HOWARD BEACH,NB_HUGUENOT,NB_HUNTS POINT,NB_INWOOD,NB_JACKSON HEIGHTS,NB_JAMAICA,NB_JAMAICA BAY,NB_JAMAICA ESTATES,NB_JAMAICA HILLS,NB_JAVITS CENTER,NB_KENSINGTON,NB_KEW GARDENS,NB_KINGSBRIDGE HTS/UNIV HTS,NB_KINGSBRIDGE/JEROME PARK,NB_KIPS BAY,NB_LAURELTON,NB_LITTLE ITALY,NB_LITTLE NECK,NB_LIVINGSTON,NB_LONG ISLAND CITY,NB_LOWER EAST SIDE,NB_MADISON,NB_MANHATTAN BEACH,NB_MANHATTAN VALLEY,NB_MANOR HEIGHTS,NB_MARINE PARK,NB_MARINERS HARBOR,NB_MASPETH,NB_MELROSE/CONCOURSE,NB_MIDDLE VILLAGE,NB_MIDLAND BEACH,NB_MIDTOWN CBD,NB_MIDTOWN EAST,NB_MIDTOWN WEST,NB_MIDWOOD,NB_MILL BASIN,NB_MORNINGSIDE HEIGHTS,NB_MORRIS PARK/VAN NEST,NB_MORRISANIA/LONGWOOD,NB_MOTT HAVEN/PORT MORRIS,NB_MOUNT HOPE/MOUNT EDEN,NB_MURRAY HILL,NB_NAVY YARD,NB_NEPONSIT,NB_NEW BRIGHTON,NB_NEW BRIGHTON-ST. GEORGE,NB_NEW DORP,NB_NEW DORP-BEACH,NB_NEW DORP-HEIGHTS,NB_NEW SPRINGVILLE,NB_OAKLAND GARDENS,NB_OAKWOOD,NB_OAKWOOD-BEACH,NB_OCEAN HILL,NB_OCEAN PARKWAY-NORTH,NB_OCEAN PARKWAY-SOUTH,NB_OLD MILL BASIN,NB_OZONE PARK,NB_PARK SLOPE,NB_PARK SLOPE SOUTH,NB_PARKCHESTER,NB_PELHAM BAY,NB_PELHAM GARDENS,NB_PELHAM PARKWAY NORTH,NB_PELHAM PARKWAY SOUTH,NB_PLEASANT PLAINS,NB_PORT IVORY,NB_PORT RICHMOND,NB_PRINCES BAY,NB_PROSPECT HEIGHTS,NB_QUEENS VILLAGE,NB_RED HOOK,NB_REGO PARK,NB_RICHMOND HILL,NB_RICHMONDTOWN,NB_RICHMONDTOWN-LIGHTHS HILL,NB_RIDGEWOOD,NB_RIVERDALE,NB_ROCKAWAY PARK,NB_ROOSEVELT ISLAND,NB_ROSEBANK,NB_ROSEDALE,NB_ROSSVILLE,NB_ROSSVILLE-CHARLESTON,NB_ROSSVILLE-PORT MOBIL,NB_ROSSVILLE-RICHMOND VALLEY,NB_SCHUYLERVILLE/PELHAM BAY,NB_SEAGATE,NB_SHEEPSHEAD BAY,NB_SILVER LAKE,NB_SO. JAMAICA-BAISLEY PARK,NB_SOHO,NB_SOUNDVIEW,NB_SOUTH BEACH,NB_SOUTH JAMAICA,NB_SOUTH OZONE PARK,NB_SOUTHBRIDGE,NB_SPRING CREEK,NB_SPRINGFIELD GARDENS,NB_ST. ALBANS,NB_STAPLETON,NB_STAPLETON-CLIFTON,NB_SUNNYSIDE,NB_SUNSET PARK,NB_THROGS NECK,NB_TODT HILL,NB_TOMPKINSVILLE,NB_TOTTENVILLE,NB_TRAVIS,NB_TRIBECA,NB_UPPER EAST SIDE (59-79),NB_UPPER

In [62]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{}

In [63]:
# Split df into X and y
y = df['SALE PRICE'].copy()
X = df.drop('SALE PRICE', axis=1).copy()

In [64]:
# Scale X with a standard scaler
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [65]:
X

RESIDENTIAL UNITS  COMMERCIAL UNITS  TOTAL UNITS  LAND SQUARE FEET  \
0               0.213097         -0.018906     0.168314         -0.059242   
1               0.556753         -0.018906     0.457734         -0.040280   
2               0.281829         -0.018906     0.226198         -0.037401   
3               0.419291         -0.018906     0.341966         -0.055770   
4               1.518989         -0.018906     1.268108          0.025508   
...                  ...               ...          ...               ...   
69982           0.006904         -0.018906    -0.005337         -0.036481   
69983           0.006904         -0.018906    -0.005337         -0.033573   
69984           0.006904         -0.018906    -0.005337          0.010997   
69985          -0.130558          0.748331     0.284082          6.065523   
69986          -0.130558          0.090700    -0.063221          0.212664   

       GROSS SQUARE FEET  YEAR BUILT  TAX CLASS AT TIME OF SALE      YEAR  \
0               0.113082    0.193234                   0.464269  0.725650   
1               0.127547    0.218191                   0.464269 -1.378076   
2               0.038510    0.193234                   0.464269 -1.378076   
3               0.022615    0.231630                   0.464269 -1.378076   
4               0.606811    0.231630                   0.464269 -1.378076   
...                  ...         ...                        ...       ...   
69982          -0.044848    0.381377                  -0.832486 -1.378076   
69983          -0.052938    0.381377                  -0.832486  0.725650   
69984          -0.088937    0.241229                  -0.832486  0.725650   
69985           2.469848    0.387136                   3.057777 -1.378076   
69986          -0.051998    0.396735                   3.057777 -1.378076   

          MONTH       DAY      BR_1      BR_2      BR_3      BR_4      BR_5  \
0      0.126698  0.295594  1.961398 -0.334663 -0.723494 -0.598129 -0.306524   
1      0.706273  0.749662  1.961398 -0.334663 -0.723494 -0.598129 -0.306524   
2      1.285848  0.068561  1.961398 -0.334663 -0.723494 -0.598129 -0.306524   
3      0.706273  0.749662  1.961398 -0.334663 -0.723494 -0.598129 -0.306524   
4      1.285848 -1.066608  1.961398 -0.334663 -0.723494 -0.598129 -0.306524   
...         ...       ...       ...       ...       ...       ...       ...   
69982  1.285848  1.317246 -0.509841 -0.334663 -0.723494 -0.598129  3.262385   
69983 -0.742665  0.522628 -0.509841 -0.334663 -0.723494 -0.598129  3.262385   
69984  0.126698 -1.293641 -0.509841 -0.334663 -0.723494 -0.598129  3.262385   
69985  1.575635  0.522628 -0.509841 -0.334663 -0.723494 -0.598129  3.262385   
69986  0.996060  1.203729 -0.509841 -0.334663 -0.723494 -0.598129  3.262385   

       NB_AIRPORT LA GUARDIA  NB_ALPHABET CITY  NB_ANNADALE  NB_ARDEN HEIGHTS  \
0                  -0.010001         21.722917    -0.043635         -0.059027   
1                  -0.010001         21.722917    -0.043635         -0.059027   
2                  -0.010001         21.722917    -0.043635         -0.059027   
3                  -0.010001         21.722917    -0.043635         -0.059027   
4                  -0.010001         21.722917    -0.043635         -0.059027   
...                      ...               ...          ...               ...   
69982              -0.010001         -0.046034    -0.043635         -0.059027   
69983              -0.010001         -0.046034    -0.043635         -0.059027   
69984              -0.010001         -0.046034    -0.043635         -0.059027   
69985              -0.010001         -0.046034    -0.043635         -0.059027   
69986              -0.010001         -0.046034    -0.043635         -0.059027   

       NB_ARROCHAR  NB_ARROCHAR-SHORE ACRES  NB_ARVERNE  NB_ASTORIA  \
0        -0.021051                -0.016907   -0.045564   -0.092363   
1        -0.021051                -0.016907   -0.045564   -0.092363   
2        -0.021051    

### Training

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [68]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [69]:
X_train.shape, X_val.shape, X_test.shape

((55989, 833), (13998, 833), (20997, 833))

In [77]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [71]:
dtrain

In [72]:
params = {'learning_rate': 0.001, 'max_depth': 6, 'lambda': 0.01}

model = xgb.train(params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:5476558.23146
[1]	eval-rmse:5473630.68273
[2]	eval-rmse:5470715.50378
[3]	eval-rmse:5467812.55422
[4]	eval-rmse:5464921.93576
[5]	eval-rmse:5462043.62792
[6]	eval-rmse:5459177.51024
[7]	eval-rmse:5456323.47002
[8]	eval-rmse:5453481.73598
[9]	eval-rmse:5450652.20110
[10]	eval-rmse:5447834.82640
[11]	eval-rmse:5445029.68068
[12]	eval-rmse:5442236.58520
[13]	eval-rmse:5439455.64510
[14]	eval-rmse:5436686.77894
[15]	eval-rmse:5433930.02247
[16]	eval-rmse:5431185.35928
[17]	eval-rmse:5428452.80620
[18]	eval-rmse:5425732.17792
[19]	eval-rmse:5423023.58284
[20]	eval-rmse:5420327.04525
[21]	eval-rmse:5417642.52059
[22]	eval-rmse:5414969.97788
[23]	eval-rmse:5412309.34664
[24]	eval-rmse:5409660.75515
[25]	eval-rmse:5407023.95524
[26]	eval-rmse:5404399.16702
[27]	eval-rmse:5401786.30170
[28]	eval-rmse:5399173.99240
[29]	eval-rmse:5396584.67024
[30]	eval-rmse:5393996.04732
[31]	eval-rmse:5392335.25674
[32]	eval-rmse:5389776.75637
[33]	eval-rmse:5387230.17465
[34]	eval-rmse:5385589.1

In [78]:
y_true = np.array(y_test)
y_pred = model.predict(dtest)

In [79]:
len(y_true), len(y_pred)

(20997, 20997)

In [80]:
print("Model R^2 Score: {:.4f}".format(r2_score(y_true, y_pred)))

Model R^2 Score: 0.3057
